In [91]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

1.	Valentino – PNW regulated market - Merchant
2.	Mantero – MISO - Merchant
3.	Howling Gale – MISO - Merchant
4.	Ventus Village – PJM – Merchant
5.	Salmon Valley – ERCOT - Merchant


In [120]:
df_salmon = pd.read_csv('./../salmon_valley.csv')

In [121]:
df_salmon.head()

,Timestamp,Year,Month,Day,Hour Ending,Maximum energy deliverable before curtailment (MWh),Energy actually delivered (MWh),Curtailment (MWh),Price ($/MWh) at the nodal level in the Real Time market
0,1/1/2023 1:00,2023,1,1,1,0.0,0.0,0,15.49
1,1/1/2023 2:00,2023,1,1,2,18.3,10.3,8,8.27
2,1/1/2023 3:00,2023,1,1,3,8.1,0.1,8,6.88
3,1/1/2023 4:00,2023,1,1,4,71.6,63.6,8,2.28
4,1/1/2023 5:00,2023,1,1,5,42.1,34.1,8,-1.52


In [122]:
df_gen_sun = pd.read_csv("./ercot_gen_sun-wnd_hr_2023.csv", header=3)

In [123]:
df_gen_sun.head()

,UTC Timestamp (Interval Ending),Local Timestamp Central Time (Interval Beginning),Local Timestamp Central Time (Interval Ending),Local Date,Hour Number,ERCOT Total Solar Generation (MW),ERCOT Total Wind Generation (MW),Coastal Wind Generation (MW),North Wind Generation (MW),Panhandle Wind Generation (MW),South Wind Generation (MW),West Wind Generation (MW)
0,2023-01-01 07:00:00,2023-01-01 00:00:00,2023-01-01 01:00:00,2023-01-01,1,0.45,21752.92,1569.53,1807.95,3453.99,2292.42,12629.03
1,2023-01-01 08:00:00,2023-01-01 01:00:00,2023-01-01 02:00:00,2023-01-01,2,0.46,21569.51,1365.66,2016.15,3417.16,1874.42,12896.12
2,2023-01-01 09:00:00,2023-01-01 02:00:00,2023-01-01 03:00:00,2023-01-01,3,0.44,21035.50,1374.09,1990.95,3163.17,1877.68,12629.61
3,2023-01-01 10:00:00,2023-01-01 03:00:00,2023-01-01 04:00:00,2023-01-01,4,0.46,20595.37,1530.15,2016.64,3159.40,1763.86,12125.32
4,2023-01-01 11:00:00,2023-01-01 04:00:00,2023-01-01 05:00:00,2023-01-01,5,0.45,20387.59,1717.90,2031.64,3165.52,1313.13,12159.40


In [124]:
df_gen_sun["Hour Number"].unique()
df_gen_sun["Hour Ending"] = df_gen_sun["Hour Number"]
print(df_gen_sun["Hour Ending"].unique())   

[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25]


In [125]:
# print where Hour Number 25
df_gen_sun[df_gen_sun["Hour Number"] == 25].head()

local_time_beginning = df_gen_sun["Local Timestamp Central Time (Interval Beginning)"]

# find the hour in the local time beginning, the local time is YYYY-MM-DD HH:MM:SS
df_gen_sun["Hour Ending"] = local_time_beginning.str.split(" ").str[1].str.split(":").str[0].astype(int) + 1

# Timestamp should be 
# Month/Day/Year Hour:Minute
# replace - with / and add hour
df_gen_sun["Timestamp"] = local_time_beginning.str.split(" ").str[0] + " " + df_gen_sun["Hour Ending"].astype(str) + ":00"
df_gen_sun["Timestamp"] = df_gen_sun["Timestamp"].str.replace("-", "/")


df_gen_sun["Year"] = local_time_beginning.str.split(" ").str[0].str.split("-").str[0]
# remove the starting 0 in the month
df_gen_sun["Month"] = local_time_beginning.str.split(" ").str[0].str.split("-").str[1].str.lstrip("0")
df_gen_sun["Day"] = local_time_beginning.str.split(" ").str[0].str.split("-").str[2].str.lstrip("0")

# make year month day as int
df_gen_sun["Year"] = df_gen_sun["Year"].astype(int)
df_gen_sun["Month"] = df_gen_sun["Month"].astype(int)
df_gen_sun["Day"] = df_gen_sun["Day"].astype(int)

rows_to_remove = ["UTC Timestamp (Interval Ending)", "Local Timestamp Central Time (Interval Beginning)", "Local Timestamp Central Time (Interval Ending)", "Local Date", "Hour Number"]

In [126]:
def make_timestamp(df):
    local_time_beginning = df["Local Timestamp Central Time (Interval Beginning)"]
    df["Hour Ending"] = local_time_beginning.str.split(" ").str[1].str.split(":").str[0].astype(int) + 1
    df["Timestamp"] = local_time_beginning.str.split(" ").str[0] + " " + df["Hour Ending"].astype(str) + ":00"
    df["Timestamp"] = df["Timestamp"].str.replace("-", "/")
    df["Year"] = local_time_beginning.str.split(" ").str[0].str.split("-").str[0]
    df["Month"] = local_time_beginning.str.split(" ").str[0].str.split("-").str[1].str.lstrip("0")
    df["Day"] = local_time_beginning.str.split(" ").str[0].str.split("-").str[2].str.lstrip("0")
    df["Year"] = df["Year"].astype(int)
    df["Month"] = df["Month"].astype(int)
    df["Day"] = df["Day"].astype(int)
    return df

In [127]:
df_gen_sun.head()

,UTC Timestamp (Interval Ending),Local Timestamp Central Time (Interval Beginning),Local Timestamp Central Time (Interval Ending),Local Date,Hour Number,ERCOT Total Solar Generation (MW),ERCOT Total Wind Generation (MW),Coastal Wind Generation (MW),North Wind Generation (MW),Panhandle Wind Generation (MW),South Wind Generation (MW),West Wind Generation (MW),Hour Ending,Timestamp,Year,Month,Day
0,2023-01-01 07:00:00,2023-01-01 00:00:00,2023-01-01 01:00:00,2023-01-01,1,0.45,21752.92,1569.53,1807.95,3453.99,2292.42,12629.03,1,2023/01/01 1:00,2023,1,1
1,2023-01-01 08:00:00,2023-01-01 01:00:00,2023-01-01 02:00:00,2023-01-01,2,0.46,21569.51,1365.66,2016.15,3417.16,1874.42,12896.12,2,2023/01/01 2:00,2023,1,1
2,2023-01-01 09:00:00,2023-01-01 02:00:00,2023-01-01 03:00:00,2023-01-01,3,0.44,21035.50,1374.09,1990.95,3163.17,1877.68,12629.61,3,2023/01/01 3:00,2023,1,1
3,2023-01-01 10:00:00,2023-01-01 03:00:00,2023-01-01 04:00:00,2023-01-01,4,0.46,20595.37,1530.15,2016.64,3159.40,1763.86,12125.32,4,2023/01/01 4:00,2023,1,1
4,2023-01-01 11:00:00,2023-01-01 04:00:00,2023-01-01 05:00:00,2023-01-01,5,0.45,20387.59,1717.90,2031.64,3165.52,1313.13,12159.40,5,2023/01/01 5:00,2023,1,1


In [128]:
df_gen_sun_copy = df_gen_sun.copy()

df_gen_sun_copy.drop(columns=rows_to_remove, inplace=True)

In [129]:
df_gen_sun_copy.head()

,ERCOT Total Solar Generation (MW),ERCOT Total Wind Generation (MW),Coastal Wind Generation (MW),North Wind Generation (MW),Panhandle Wind Generation (MW),South Wind Generation (MW),West Wind Generation (MW),Hour Ending,Timestamp,Year,Month,Day
0,0.45,21752.92,1569.53,1807.95,3453.99,2292.42,12629.03,1,2023/01/01 1:00,2023,1,1
1,0.46,21569.51,1365.66,2016.15,3417.16,1874.42,12896.12,2,2023/01/01 2:00,2023,1,1
2,0.44,21035.50,1374.09,1990.95,3163.17,1877.68,12629.61,3,2023/01/01 3:00,2023,1,1
3,0.46,20595.37,1530.15,2016.64,3159.40,1763.86,12125.32,4,2023/01/01 4:00,2023,1,1
4,0.45,20387.59,1717.90,2031.64,3165.52,1313.13,12159.40,5,2023/01/01 5:00,2023,1,1


In [130]:
index_cols = ["Year", "Month", "Day", "Hour Ending"]

In [133]:
# merge 
df_gen_sun_copy.drop(columns=["Timestamp"], inplace=True)
df_merge = pd.merge(df_gen_sun_copy, df_salmon, on=index_cols)

In [134]:
df_merge.head()

,ERCOT Total Solar Generation (MW),ERCOT Total Wind Generation (MW),Coastal Wind Generation (MW),North Wind Generation (MW),Panhandle Wind Generation (MW),South Wind Generation (MW),West Wind Generation (MW),Hour Ending,Year,Month,Day,Timestamp,Maximum energy deliverable before curtailment (MWh),Energy actually delivered (MWh),Curtailment (MWh),Price ($/MWh) at the nodal level in the Real Time market
0,0.45,21752.92,1569.53,1807.95,3453.99,2292.42,12629.03,1,2023,1,1,1/1/2023 1:00,0.0,0.0,0,15.49
1,0.46,21569.51,1365.66,2016.15,3417.16,1874.42,12896.12,2,2023,1,1,1/1/2023 2:00,18.3,10.3,8,8.27
2,0.44,21035.50,1374.09,1990.95,3163.17,1877.68,12629.61,3,2023,1,1,1/1/2023 3:00,8.1,0.1,8,6.88
3,0.46,20595.37,1530.15,2016.64,3159.40,1763.86,12125.32,4,2023,1,1,1/1/2023 4:00,71.6,63.6,8,2.28
4,0.45,20387.59,1717.90,2031.64,3165.52,1313.13,12159.40,5,2023,1,1,1/1/2023 5:00,42.1,34.1,8,-1.52


In [135]:
df_load_act = pd.read_csv("./ercot_load_act_hr_2023.csv", header=3)
df_load_for = pd.read_csv("./ercot_load_for_hr_2023.csv", header=3)

In [136]:
df_load_act = make_timestamp(df_load_act)
df_load_for = make_timestamp(df_load_for)

In [137]:
df_load_act.head()
# col: TOTAL Actual Load (MW)

,UTC Timestamp (Interval Ending),Local Timestamp Central Time (Interval Beginning),Local Timestamp Central Time (Interval Ending),Local Date,Hour Number,TOTAL Actual Load (MW),Coast Actual Load (MW),East Actual Load (MW),Far West Actual Load (MW),North Actual Load (MW),North Central Actual Load (MW),Southern Actual Load (MW),South Central Actual Load (MW),West Actual Load (MW),Hour Ending,Timestamp,Year,Month,Day
0,2023-01-01 07:00:00,2023-01-01 00:00:00,2023-01-01 01:00:00,2023-01-01,1,35845.13,8599.43,1247.62,5287.70,762.30,10114.29,2875.20,5846.99,1111.60,1,2023/01/01 1:00,2023,1,1
1,2023-01-01 08:00:00,2023-01-01 01:00:00,2023-01-01 02:00:00,2023-01-01,2,35263.44,8466.87,1222.11,5246.51,744.58,9929.96,2796.42,5747.88,1109.11,2,2023/01/01 2:00,2023,1,1
2,2023-01-01 09:00:00,2023-01-01 02:00:00,2023-01-01 03:00:00,2023-01-01,3,34613.53,8286.36,1202.47,5245.36,737.91,9719.65,2724.95,5603.94,1092.88,3,2023/01/01 3:00,2023,1,1
3,2023-01-01 10:00:00,2023-01-01 03:00:00,2023-01-01 04:00:00,2023-01-01,4,34236.02,8138.62,1204.02,5265.49,753.20,9571.90,2691.34,5536.30,1075.16,4,2023/01/01 4:00,2023,1,1
4,2023-01-01 11:00:00,2023-01-01 04:00:00,2023-01-01 05:00:00,2023-01-01,5,34088.96,8057.78,1214.01,5251.73,766.62,9581.12,2588.30,5535.74,1093.66,5,2023/01/01 5:00,2023,1,1


In [138]:
df_load_for.head()
# col: SystemTotal Forecast Load (MW)

,UTC Timestamp (Interval Ending),Local Timestamp Central Time (Interval Beginning),Local Timestamp Central Time (Interval Ending),Local Date,Hour Number,SystemTotal Forecast Load (MW),Houston Forecast Load (MW),North Forecast Load (MW),South Forecast Load (MW),West Forecast Load (MW),Hour Ending,Timestamp,Year,Month,Day
0,2023-01-22 07:00:00,2023-01-22 00:00:00,2023-01-22 01:00:00,2023-01-22,1,41479.637390,9884.130226,13892.669750,10783.429205,6919.408209,1,2023/01/22 1:00,2023,1,22
1,2023-01-22 08:00:00,2023-01-22 01:00:00,2023-01-22 02:00:00,2023-01-22,2,40796.827637,9612.301703,13763.951039,10527.050248,6893.524646,2,2023/01/22 2:00,2023,1,22
2,2023-01-22 09:00:00,2023-01-22 02:00:00,2023-01-22 03:00:00,2023-01-22,3,40661.955627,9412.575441,13821.788013,10467.253638,6960.338536,3,2023/01/22 3:00,2023,1,22
3,2023-01-22 10:00:00,2023-01-22 03:00:00,2023-01-22 04:00:00,2023-01-22,4,40817.816406,9415.155129,13979.117020,10483.750527,6939.793730,4,2023/01/22 4:00,2023,1,22
4,2023-01-22 11:00:00,2023-01-22 04:00:00,2023-01-22 05:00:00,2023-01-22,5,41342.678955,9413.484084,14312.787270,10650.848057,6965.559545,5,2023/01/22 5:00,2023,1,22


In [139]:
df_merge.head()

,ERCOT Total Solar Generation (MW),ERCOT Total Wind Generation (MW),Coastal Wind Generation (MW),North Wind Generation (MW),Panhandle Wind Generation (MW),South Wind Generation (MW),West Wind Generation (MW),Hour Ending,Year,Month,Day,Timestamp,Maximum energy deliverable before curtailment (MWh),Energy actually delivered (MWh),Curtailment (MWh),Price ($/MWh) at the nodal level in the Real Time market
0,0.45,21752.92,1569.53,1807.95,3453.99,2292.42,12629.03,1,2023,1,1,1/1/2023 1:00,0.0,0.0,0,15.49
1,0.46,21569.51,1365.66,2016.15,3417.16,1874.42,12896.12,2,2023,1,1,1/1/2023 2:00,18.3,10.3,8,8.27
2,0.44,21035.50,1374.09,1990.95,3163.17,1877.68,12629.61,3,2023,1,1,1/1/2023 3:00,8.1,0.1,8,6.88
3,0.46,20595.37,1530.15,2016.64,3159.40,1763.86,12125.32,4,2023,1,1,1/1/2023 4:00,71.6,63.6,8,2.28
4,0.45,20387.59,1717.90,2031.64,3165.52,1313.13,12159.40,5,2023,1,1,1/1/2023 5:00,42.1,34.1,8,-1.52


In [140]:
df_load_for_only = df_load_for[index_cols + ["SystemTotal Forecast Load (MW)"]]
# drop Timestamp column
df_merge_for = pd.merge(df_merge, df_load_for_only, on=index_cols)
df_merge_for.head()

,ERCOT Total Solar Generation (MW),ERCOT Total Wind Generation (MW),Coastal Wind Generation (MW),North Wind Generation (MW),Panhandle Wind Generation (MW),South Wind Generation (MW),West Wind Generation (MW),Hour Ending,Year,Month,Day,Timestamp,Maximum energy deliverable before curtailment (MWh),Energy actually delivered (MWh),Curtailment (MWh),Price ($/MWh) at the nodal level in the Real Time market,SystemTotal Forecast Load (MW)
0,0.38,16735.78,120.30,1881.64,2395.53,331.65,12006.66,1,2023,1,22,1/22/2023 1:00,54.5,54.5,0,21.69,41479.637390
1,0.39,15817.67,329.69,2024.64,2576.16,324.09,10563.09,2,2023,1,22,1/22/2023 2:00,58.1,58.1,0,18.03,40796.827637
2,0.37,15018.62,581.15,1881.46,2547.31,630.77,9377.93,3,2023,1,22,1/22/2023 3:00,47.9,47.9,0,9.40,40661.955627
3,0.36,14402.60,1175.90,1756.53,2463.41,1081.30,7925.46,4,2023,1,22,1/22/2023 4:00,44.2,36.2,8,8.52,40817.816406
4,0.37,13801.15,1492.87,1584.70,2161.81,1484.59,7077.18,5,2023,1,22,1/22/2023 5:00,39.6,39.6,0,12.58,41342.678955


In [141]:
df_load_act[index_cols + ["TOTAL Actual Load (MW)"]].head()

,Year,Month,Day,Hour Ending,TOTAL Actual Load (MW)
0,2023,1,1,1,35845.13
1,2023,1,1,2,35263.44
2,2023,1,1,3,34613.53
3,2023,1,1,4,34236.02
4,2023,1,1,5,34088.96


In [142]:
df_load_act_only = df_load_act[index_cols + ["TOTAL Actual Load (MW)"]]
df_merge_act = pd.merge(df_merge, df_load_act_only, on=index_cols)
df_merge_act.head()

,ERCOT Total Solar Generation (MW),ERCOT Total Wind Generation (MW),Coastal Wind Generation (MW),North Wind Generation (MW),Panhandle Wind Generation (MW),South Wind Generation (MW),West Wind Generation (MW),Hour Ending,Year,Month,Day,Timestamp,Maximum energy deliverable before curtailment (MWh),Energy actually delivered (MWh),Curtailment (MWh),Price ($/MWh) at the nodal level in the Real Time market,TOTAL Actual Load (MW)
0,0.45,21752.92,1569.53,1807.95,3453.99,2292.42,12629.03,1,2023,1,1,1/1/2023 1:00,0.0,0.0,0,15.49,35845.13
1,0.46,21569.51,1365.66,2016.15,3417.16,1874.42,12896.12,2,2023,1,1,1/1/2023 2:00,18.3,10.3,8,8.27,35263.44
2,0.44,21035.50,1374.09,1990.95,3163.17,1877.68,12629.61,3,2023,1,1,1/1/2023 3:00,8.1,0.1,8,6.88,34613.53
3,0.46,20595.37,1530.15,2016.64,3159.40,1763.86,12125.32,4,2023,1,1,1/1/2023 4:00,71.6,63.6,8,2.28,34236.02
4,0.45,20387.59,1717.90,2031.64,3165.52,1313.13,12159.40,5,2023,1,1,1/1/2023 5:00,42.1,34.1,8,-1.52,34088.96


In [143]:
# merge both into one
df_merge_loads = pd.merge(df_merge_act, df_load_for_only, on=index_cols)

In [144]:
df_merge_loads.head()

,ERCOT Total Solar Generation (MW),ERCOT Total Wind Generation (MW),Coastal Wind Generation (MW),North Wind Generation (MW),Panhandle Wind Generation (MW),South Wind Generation (MW),West Wind Generation (MW),Hour Ending,Year,Month,Day,Timestamp,Maximum energy deliverable before curtailment (MWh),Energy actually delivered (MWh),Curtailment (MWh),Price ($/MWh) at the nodal level in the Real Time market,TOTAL Actual Load (MW),SystemTotal Forecast Load (MW)
0,0.38,16735.78,120.30,1881.64,2395.53,331.65,12006.66,1,2023,1,22,1/22/2023 1:00,54.5,54.5,0,21.69,39644.14,41479.637390
1,0.39,15817.67,329.69,2024.64,2576.16,324.09,10563.09,2,2023,1,22,1/22/2023 2:00,58.1,58.1,0,18.03,39141.07,40796.827637
2,0.37,15018.62,581.15,1881.46,2547.31,630.77,9377.93,3,2023,1,22,1/22/2023 3:00,47.9,47.9,0,9.40,38835.83,40661.955627
3,0.36,14402.60,1175.90,1756.53,2463.41,1081.30,7925.46,4,2023,1,22,1/22/2023 4:00,44.2,36.2,8,8.52,38917.22,40817.816406
4,0.37,13801.15,1492.87,1584.70,2161.81,1484.59,7077.18,5,2023,1,22,1/22/2023 5:00,39.6,39.6,0,12.58,39547.24,41342.678955


In [74]:
# save the merged data
df_merge_loads.to_csv("./ercot_merged.csv", index=False)